In [ ]:
import random
import subprocess

def process_video(input_file, output_file):
    """
    使用 FFmpeg 对视频进行进一步处理以规避重复内容审核：
    1. 移除元数据。
    2. 调整分辨率到 1280x720，随机裁剪或加边框。
    3. 调整帧率到 30fps。
    4. 添加动态亮度、对比度调整和随机噪声。
    5. 添加透明水印。
    6. 处理音频：调整音频速度和混响。
    """
    # 随机位置水印
    watermark_x = random.randint(0, 1280 - 200)  # 随机 X 位置
    watermark_y = random.randint(0, 720 - 50)   # 随机 Y 位置
    watermark_opacity = random.uniform(0.1, 0.3)  # 随机透明度

    # 随机色彩调整
    brightness = random.uniform(-0.05, 0.05)  # 随机亮度
    contrast = random.uniform(0.95, 1.05)     # 随机对比度
    saturation = random.uniform(0.95, 1.05)   # 随机饱和度

    # 创建 FFmpeg 命令
    command = [
        "ffmpeg", "-i", input_file,
        # 移除元数据
        "-map_metadata", "-1",
        # 视频滤镜：调整分辨率、亮度对比、随机裁剪、加边框、添加噪声和水印
        "-vf", f"scale=1280:720,eq=brightness={brightness}:contrast={contrast}:saturation={saturation},"
               f"noise=alls=10:allf=t+u,"
               f"drawtext=text='0.007':fontcolor=white@{watermark_opacity}:fontsize=24:x={watermark_x}:y={watermark_y}",
        # 设置帧率
        "-r", "30",
        # 视频编码器和参数
        "-c:v", "libx264", "-crf", "18", "-b:v", "3M", "-preset", "slow",
        # 音频滤镜：调整音频速度和音量
        "-filter:a", "atempo=1.05,volume=1.1",
        # 音频编码器
        "-c:a", "aac",
        # 视频剪辑（轻微调整）
        "-ss", "00:00:01",  # 从 5 秒开始
        "-t", "00:03:00",   # 保留 3 分钟
        # 轻微加速
        "-filter:v", "setpts=1.05*PTS",  # 视频加速 5%
        "-filter:a", "atempo=1.05",      # 音频同步
        # 输出文件
        output_file
    ]

    # 执行 FFmpeg 命令
    try:
        subprocess.run(command, check=True)
        print(f"视频处理完成，保存为: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"处理视频时出错: {e}")

# 示例用法
process_video("input_video.mp4", "output_video.mp4")


In [ ]:
import os

# 创建 processed 文件夹（如果不存在）
processed_dir = "./processed"
os.makedirs(processed_dir, exist_ok=True)

# 找到所有 .webm 文件并逐个处理
webm_files = glob.glob("./downloads/*.webm")
for input_file in webm_files:
    output_file = os.path.join(processed_dir, os.path.basename(input_file).replace(".webm", ".mp4"))
    process_video(input_file, output_file)